#Jupyter file to prototype generating Weil codes

In [5]:
import numpy as np
import pandas as pd

In [6]:
df = pd.read_csv("L1Cp.csv",
                 dtype={"PRN": int,
                        "w_pilot": int,
                        "p_pilot": int,
                        "Initial_pilot_code": str,
                        "Final_pilot_code":str
                       }
                )

In [7]:
prn_value = 11
result = df.loc[df["PRN"] == prn_value, "w_pilot"].iloc[0]
start_string = df.loc[df["PRN"] == prn_value, "Initial_pilot_code"].iloc[0]
print (result)
print (start_string)

5093
04733076


In [8]:
print (df.dtypes)
print (df.head())
l1c_table = df.set_index("PRN", verify_integrity=True)

PRN                    int64
w_pilot                int64
p_pilot                int64
Initial_pilot_code    object
Final_pilot_code      object
dtype: object
   PRN  w_pilot  p_pilot Initial_pilot_code Final_pilot_code
0    1     5111      412           05752067         20173742
1    4     5106      303           72125121         71435437
2   11     5093      253           04733076         15210113
3   14     5081       66           07476042         46623624
4   18     5054     5211           73202225         14040513


In [9]:
w = l1c_table.at[int(11), "w_pilot"]
p = l1c_table.at[int(11), "p_pilot"]
print (w,p)

5093 253


In [10]:
def jacobi(n: int, k: int) -> int:
    """
    Computes the Jacobi symbol (n/k) for odd k > 0.
    Returns -1, 0, or +1.
    """
    assert k > 0 and k % 2 == 1
    n = n % k
    t = 1
    while n != 0:
        # factor out powers of 2 from n
        while n % 2 == 0:
            n //= 2
            r = k % 8
            if r == 3 or r == 5:
                t = -t
        # quadratic reciprocity step
        n, k = k, n
        if (n % 4 == 3) and (k % 4 == 3):
            t = -t
        n = n % k
    return t if k == 1 else 0

In [11]:
for a in range(7):
    print(a, jacobi(a, 7))
# expected: 0,+1,+1,−1,+1,−1,−1
for a in range(11):
    print(a, jacobi(a, 11))
# check against known residues: {0,1,3,4,5,9}


0 0
1 1
2 1
3 -1
4 1
5 -1
6 -1
0 0
1 1
2 -1
3 1
4 1
5 1
6 -1
7 -1
8 -1
9 1
10 -1


In [12]:
def jacobi_sequence(k: int, start: int = 1, length: int | None = None, map_zero_to: int = +1) -> np.ndarray:
    """
    Build a Jacobi-based ±1 sequence over modulus k (odd).
      - start: first n to evaluate (typical: 1)
      - For GPS, we need start = 0
      - length: number of terms; default = k - start (so 1..k-1 if start=1)
      - map_zero_to: map any 0 results to ±1 to keep a binary alphabet
    """
    assert k > 0 and k % 2 == 1
    if length is None:
        length = k - start
    ns = np.arange(start, start + length, dtype=np.int64)

    # Vectorize the scalar jacobi() while keeping identical semantics.
    vjac = np.vectorize(lambda n: jacobi(int(n), k), otypes=[int])
    vals = vjac(ns).astype(np.int8)

    if map_zero_to not in (-1, 0, +1):
        raise ValueError("map_zero_to must be one of 0, +1, -1")
    vals[vals == 0] = map_zero_to

    # vals is in {-1, +1}. If you need {0,1}, use: (vals > 0).astype(np.uint8)
    return vals

In [13]:
#k = 101  # must be odd
k = 11
chips_pm1 = jacobi_sequence(k, start=0, map_zero_to=0)              # ±1 chips
#chips_01  = (chips_pm1 > 0).astype(np.uint8)  # 0/1 chips if needed
print(chips_pm1, "… len =", chips_pm1.size)

[ 0  1 -1  1  1  1 -1 -1 -1  1 -1] … len = 11


In [14]:
print (chips_pm1[:32], "... len =", chips_pm1.size)

[ 0  1 -1  1  1  1 -1 -1 -1  1 -1] ... len = 11


In [15]:
def l1c_pilot_weil(w: int, p: int, *, k: int, N0: int, alphabet: str = "pm1") -> np.ndarray:
    """
    Generate an L1C Pilot spreading sequence via Weil/Jacobi construction.

    Parameters
    ----------
    w : int
        Weil index for the PRN.
    p : int
        Insertion point (1-based).
    k : int
        Odd modulus from the ICD table (e.g. 10223).
    N0 : int
        Number of chips (pilot sequence length).
    alphabet : {"pm1", "01"}
        Output alphabet: ±1 (pm1) or {0,1}.

    Returns
    -------
    chips : np.ndarray
        1D array of length N0 in the requested alphabet.
    """

    # quadratic residue indicator
    idx = np.arange(k, dtype=np.int64)
    vjac = np.vectorize(lambda n: jacobi(int(n), k, start=0, map_zero_to=0))
    resid01 = (vjac(idx) == 1).astype(np.uint8)

    # Weil code: XOR with shifted copy
    resid01_shift = np.roll(resid01, w % k)
    weil01 = resid01 ^ resid01_shift

    # Extract N0 chips starting at insertion point p (1-based)
    start = (p - 1) % k
    if N0 <= k - start:
        out01 = weil01[start:start + N0].copy()
    else:
        out01 = np.concatenate((weil01[start:], weil01[:N0 - (k - start)]))

    if alphabet == "01":
        return out01
    elif alphabet == "pm1":
        return (out01 * 2 - 1).astype(np.int8)
    else:
        raise ValueError("alphabet must be 'pm1' or '01'")


In [16]:


# Scalar jacobi(n,k) — your tested version goes here

def jacobi_full_sequence(k: int) -> np.ndarray:
    """L(n) for n=0..k-1 in {-1,0,+1}. Use this for GPS L1C construction."""
    return np.array([jacobi(n, k) for n in range(k)], dtype=np.int8)

def l1c_pilot_from_symbols(W: int, P: int, *,
                           k: int = 10223, N0: int = 10230,
                           p_base: int = 1, shift: str = "right",
                           out: str = "01",
                           L: np.ndarray | None = None) -> np.ndarray:
    """
    Build L1C pilot using L(n) in {-1,0,+1}:
      w(n) = (1 - L(n)*L(n+W)) / 2  in {0,1}
    - p_base=1 because ICD tables list P as 1-based.
    - shift='right' means L(n+W); use 'left' for L(n-W).
    - out='01' or 'pm1'.
    - Optionally pass a precomputed L to avoid recomputing per PRN.
    """
    if L is None:
        L = jacobi_full_sequence(k)  # includes n=0 term == 0
    Wm = W % k
    Ls = np.roll(L, Wm if shift == "right" else -Wm)

    # Weil bits from symbol product (keep zeros intact here)
    prod = (L.astype(np.int16) * Ls.astype(np.int16))      # {-1,0,+1}
    weil01 = ((1 - prod) // 2).astype(np.uint8)            # +1->0, 0/-1->1

    # Extract N0 chips starting at insertion point P (1-based default)
    start = (P - p_base) % k
    if N0 <= k - start:
        out01 = weil01[start:start + N0].copy()
    else:
        tail = k - start
        out01 = np.concatenate((weil01[start:], weil01[:N0 - tail]))

    if out == "01":
        return out01
    elif out == "pm1":
        return (out01 * 2 - 1).astype(np.int8)
    else:
        raise ValueError("out must be '01' or 'pm1'")


In [17]:
print (jacobi_full_sequence(11))

[ 0  1 -1  1  1  1 -1 -1 -1  1 -1]


In [121]:
import numpy as np

def jacobi(n: int, k: int) -> int:
    """
    Computes the Jacobi symbol (n/k) for odd k > 0.
    Returns -1, 0, or +1.
    """
    assert k > 0 and k % 2 == 1
    n = n % k
    t = 1
    while n != 0:
        # factor out powers of 2 from n
        while n % 2 == 0:
            n //= 2
            r = k % 8
            if r == 3 or r == 5:
                t = -t
        # quadratic reciprocity step
        n, k = k, n
        if (n % 4 == 3) and (k % 4 == 3):
            t = -t
        n = n % k
    return t if k == 1 else 0
#
#
#

def jacobi_full_sequence(k: int) -> np.ndarray:
    """
    Compute the Jacobi sequence L(n) for n = 0..k-1.
    Values returned are in {-1, 0, +1}.

    Jacobi is a more general version of the Legendre sequence.
    Legendre sequence results when Jacobi called with a prime number.

    Parameters
    ----------
    k : int
        Odd modulus. For GPS L1C, k = 10223.

    Returns
    -------
    L : np.ndarray
        Array of shape (k,), with entries in {-1,0,+1}.
    """
    values = []
    for n in range(k):
        values.append(jacobi(n, k))  # uses your scalar jacobi() function
    return np.array(values, dtype=np.int8)
#
#
#

'''
Commented out for now

def weil_sequence(W: int, P:int) -> np.ndarray:
    k = 10223
    L = jacobi_full_sequence (k)
    L_shift = np.roll (L, -1*W)
    w = (1 - (L * L_shift)) / 2
    expansion = np.array([-1, 1, 1, -1, 1, -1, -1])
    break_index = P-1
    if not (0 <= break_index < k) or not (0 < break_index <= k):
        raise ValueError("P out of range for 7-chip replacement.")
    weil = np.concatenate((w[:break_index], expansion, w[break_index:]))
    binary_result = (weil > 0).astype(int)
    return (binary_result)
'''

def bits_to_octal(bits01: np.ndarray, msb_first: bool = True, reverse_bits_in_group: bool = False) -> str:
    """Pack 0/1 bits to octal string, 3 bits per digit."""
    # truncate to multiple of 3 for clean comparison
    n = (bits01.size // 3) * 3
    b = bits01[:n].reshape(-1, 3)
    if reverse_bits_in_group:
        b = b[:, ::-1]
    if msb_first:
        vals = (b[:,0] << 2) | (b[:,1] << 1) | b[:,2]
    else:
        # if someone meant LSB-first groups
        vals = b[:,0] | (b[:,1] << 1) | (b[:,2] << 2)
    return "".join(str(int(v)) for v in vals)



In [127]:
def weil_sequence(W_index: int, P:int) -> np.ndarray:
    k = 10223
    L_pm1 = jacobi_full_sequence(k).astype(np.int8)   #Legendre sequence returns {-1, 0, +1}
    L_bin = (L_pm1 == 1).astype(np.uint8)             #binary Legendre sequence: map {-1, 0} to 0, +1 to 1
    W1 = W_index % k                                   #handle the edge cases
    L_shift = np.roll (L_bin, -1*W)
    w = L_bin ^ L_shift
    expansion = np.array([-1, 1, 1, -1, 1, -1, -1])
    break_index = P-1
    if not (0 <= break_index < k) or not (0 < break_index <= k):
        raise ValueError("P out of range for 7-chip replacement.")
    weil = np.concatenate((w[:break_index], expansion, w[break_index:]))
    return (weil)

In [128]:
W1 = 5111
P1 = 412
k = 10223

L10223 = jacobi_full_sequence(10223)
Ls2 = np.zeros(10223).astype(int)
for t in np.arange(0,k-1):
    shift_index = ((t+W1) % k).astype(int)
    Ls2[t] = L10223[shift_index].astype(int)
    
    
L_shift = np.roll(L10223, -1*W1)
result = (L10223 * L_shift).astype(int)

print (f'First vector:  {L10223[:24]}')
print (f'Second vector: {L_shift[:24]}')
print (f'Shift Loop:    {Ls2[:24]}')
print (f'Result:        {result[:24]}')

Leg_binary = (L10223 > 0).astype(int)
print (f'Leg_binary: {Leg_binary[:24]}')

result = (result > 0).astype(int)
print (f'Binary Result: {result[:12]}')

octal_string = bits01_to_octal(result)
print (f'Octal: {octal_string[:12]}')


weil = weil_sequence(W1, P1)
print (f'Weil Sequence: {weil [:32]}')
print (f'Length of sequence: {len(weil)}')
weil_octal = bits01_to_octal(weil)
print (f'Weil Sequence: {weil_octal[0:24]}')

First vector:  [ 0  1  1  1  1 -1  1  1  1  1 -1 -1  1 -1  1 -1  1 -1  1 -1 -1  1 -1  1]
Second vector: [-1  1  1 -1  1  1 -1 -1 -1 -1 -1  1  1  1  1 -1  1 -1 -1  1 -1 -1  1 -1]
Shift Loop:    [-1  1  1 -1  1  1 -1 -1 -1 -1 -1  1  1  1  1 -1  1 -1 -1  1 -1 -1  1 -1]
Result:        [ 0  1  1 -1  1 -1 -1 -1 -1 -1  1 -1  1 -1  1  1  1  1 -1 -1  1 -1 -1 -1]
Leg_binary: [0 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0 1 0 0 1 0 1]
Binary Result: [0 1 1 0 1 0 0 0 0 0 1 0]
Octal: 320257103772
Weil Sequence: [0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 1 1 0 0 0 0 0 0 0]
Length of sequence: 10230
Weil Sequence: 057520674005110335241407


In [120]:
k = 10223
# L in {-1,0,+1} from your jacobi_full_sequence
L_pm1 = jacobi_full_sequence(k).astype(np.int8)

# Convert to the ICD's binary Legendre: residues->1, everything else (incl. t=0) -> 0
L_bin = (L_pm1 == 1).astype(np.uint8)          # {0,1}, with L_bin[0] == 0

# Weil base sequence: XOR, using L(n+W).  Note: np.roll(..., -W) implements index (n+W) % k
W = W1 % k
w = L_bin ^ np.roll(L_bin, -W)                 # {0,1}, length 10223

# Insert the common 7-bit expansion at 1-based index p (Python index p-1)
# Use the expansion sequence specified by the ICD (same for all PRNs):
expansion = np.array([0, 1, 1, 0, 1, 0, 0], dtype=np.uint8)

p0 = p - 1                                     # p is 1-based in the tables
code = np.concatenate([w[:p0], expansion, w[p0:]])   # length 10230

print (f'code = {code[:24]}')
octal_string = bits01_to_octal(code)
print (f'Octal: {octal_string[:12]}')

code = [0 0 0 1 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1 0 1 1 1]
Octal: 057520674005


In [74]:
# Precompute L once
L10223 = jacobi_full_sequence(10223)


# Structural properties (for any prime k):
# - Exactly two positions per period have prod == 0: n=0 and n≡-W (mod k)
def check_structure(W):
    Wm = W % 10223
    Ls = np.roll(L10223, Wm)
    prod = L10223.astype(int) * Ls.astype(int)
    zeros = np.flatnonzero(prod == 0)
    assert len(zeros) == 2, f"Expected 2 zeros, got {len(zeros)} at {zeros}"
    # And weil bit is 0 iff prod==+1
    weil01 = (prod <= 0).astype(np.uint8)
    assert np.all(weil01[prod == 1] == 0)
    assert np.all(weil01[prod <= 0] == 1)

# Try with your (W,P)
W, P = 5111, 412
check_structure(W)

chips01 = l1c_pilot_from_symbols(W, P, k=10223, N0=10230, p_base=1, shift="right", out="01", L=L10223)

# If your table’s octal still doesn’t match, try just these tiny toggles:
#   shift="left"      (L(n-W) instead of L(n+W))
#   p_base=0          (if P is actually 0-based)
#   chips01 ^= 1      (if the table is complemented)
print (chips01[:32])


[1 0 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0]


In [41]:
PRN = 1
w_pilot = l1c_table.at[PRN, "w_pilot"]
p_pilot = l1c_table.at[PRN, "p_pilot"]
print (f"W = {w_pilot}, P={p_pilot}")

chips_pm1 = l1c_pilot_from_symbols (w_pilot, p_pilot)

#chips_pm1 = l1c_pilot_weil(
#    w=w_pilot,
#    p=p_pilot,
#    k=10223,
#    N0=10230,
#    alphabet="01"
#)

print(chips_pm1[:32])
print(chips_pm1.shape)

W = 5111, P=412
[1 0 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0]
(10230,)


In [ ]:
#
# Not sure if this is worth saving, but I'm not ready to delete it.

def l1c_pilot_from_symbols(W: int, P: int, *,
                           k: int = 10223, N0: int = 10230,
                           p_base: int = 1, shift: str = "right",
                           out: str = "01",
                           L: np.ndarray | None = None) -> np.ndarray:
    """
    Build L1C pilot using L(n) in {-1,0,+1}:
      w(n) = (1 - L(n)*L(n+W)) / 2  in {0,1}
    - p_base=1 because ICD tables list P as 1-based.
    - shift='right' means L(n+W); use 'left' for L(n-W).
    - out='01' or 'pm1'.
    - Optionally pass a precomputed L to avoid recomputing per PRN.
    """
    if L is None:
        L = jacobi_full_sequence(k)  # includes n=0 term == 0
    Wm = W % k
    Ls = np.roll(L, Wm if shift == "right" else -Wm)

    # Weil bits from symbol product (keep zeros intact here)
    prod = (L.astype(np.int16) * Ls.astype(np.int16))      # {-1,0,+1}
    weil01 = (prod <= 0).astype(np.uint8)            # +1->0, 0/-1->1

    # Extract N0 chips starting at insertion point P (1-based default)
    start = (P - p_base) % k
    if N0 <= k - start:
        out01 = weil01[start:start + N0].copy()
    else:
        tail = k - start
        out01 = np.concatenate((weil01[start:], weil01[:N0 - tail]))

    if out == "01":
        return out01
    elif out == "pm1":
        return (out01 * 2 - 1).astype(np.int8)
    else:
        raise ValueError("out must be '01' or 'pm1'")


In [42]:
groups = chips_pm1.reshape(-1,3)
print(groups.shape)

(3410, 3)


In [38]:
oct_digits = [str(int(''.join(map(str, g)), 2)) for g in groups]

In [22]:
print (groups[:10,:])

[[1 0 1]
 [0 0 0]
 [0 0 0]
 [1 1 0]
 [1 0 1]
 [1 0 0]
 [0 0 1]
 [1 0 0]
 [0 0 0]
 [0 0 1]]


In [23]:
#print (oct_digits.shape())
print (oct_digits[:10])

['5', '0', '0', '6', '5', '4', '1', '4', '0', '1']


In [24]:
import numpy as np
from dataclasses import dataclass

# --- Core: Jacobi, residue, Weil, slice ---
def jacobi(n: int, k: int) -> int:
    assert k > 0 and (k % 2 == 1)
    n = n % k
    t = 1
    while n != 0:
        while n % 2 == 0:
            n //= 2
            r = k % 8
            if r == 3 or r == 5:
                t = -t
        n, k = k, n
        if (n % 4 == 3) and (k % 4 == 3):
            t = -t
        n = n % k
    return t if k == 1 else 0

def residue_indicator(k: int, map_zero_to_one: bool = True) -> np.ndarray:
    """Return resid01[n] = 1 if (n/k)==+1, else 0; map Jacobi==0 to {1 or 0}."""
    idx = np.arange(k, dtype=np.int64)
    vjac = np.vectorize(lambda n: jacobi(int(n), k), otypes=[int])
    jvals = vjac(idx).astype(np.int8)
    resid01 = (jvals == 1).astype(np.uint8)
    if map_zero_to_one:
        resid01[jvals == 0] = 1
    # else: leave 0 for Jacobi==0
    return resid01

@dataclass
class L1cOptions:
    k: int = 10223             # modulus for Jacobi/Weil
    N0: int = 10230            # output length (pilot)
    map_zero_to_one: bool = False  # ICDs typically treat Jacobi==0 as 0 in indicator
    shift_direction: str = "right" # "right" => np.roll(+w); "left" => np.roll(-w)
    xor: bool = True           # True: XOR (Weil standard); False: XNOR (inversion)
    p_base: int = 1            # 1 if insertion point p is 1-based; 0 if p is 0-based
    invert: bool = False       # invert entire output after slice
    octal_msb_first: bool = True   # group bits MSB-first into octal
    reverse_bits_in_group: bool = False  # swap bit order within each 3-bit chunk

def build_l1c_pilot(w: int, p: int, opt: L1cOptions) -> np.ndarray:
    k = opt.k
    resid01 = residue_indicator(k, map_zero_to_one=opt.map_zero_to_one)

    # shifted copy
    if opt.shift_direction == "right":
        resid01_shift = np.roll(resid01, w % k)
    elif opt.shift_direction == "left":
        resid01_shift = np.roll(resid01, -(w % k))
    else:
        raise ValueError("shift_direction must be 'right' or 'left'")

    # XOR or XNOR
    if opt.xor:
        weil01 = (resid01 ^ resid01_shift).astype(np.uint8)
    else:
        weil01 = (1 - (resid01 ^ resid01_shift)).astype(np.uint8)

    # insertion point
    start = (p - opt.p_base) % k

    # slice with wrap
    N0 = opt.N0
    if N0 <= k - start:
        out01 = weil01[start:start + N0].copy()
    else:
        tail = k - start
        head = N0 - tail
        out01 = np.concatenate((weil01[start:], weil01[:head]))

    if opt.invert:
        out01 = 1 - out01
    return out01

# --- Octal helpers & pretty printing ---
def bits01_to_octal(bits01: np.ndarray, msb_first: bool = True, reverse_bits_in_group: bool = False) -> str:
    """Pack 0/1 bits to octal string, 3 bits per digit."""
    # truncate to multiple of 3 for clean comparison
    n = (bits01.size // 3) * 3
    b = bits01[:n].reshape(-1, 3)
    if reverse_bits_in_group:
        b = b[:, ::-1]
    if msb_first:
        vals = (b[:,0] << 2) | (b[:,1] << 1) | b[:,2]
    else:
        # if someone meant LSB-first groups
        vals = b[:,0] | (b[:,1] << 1) | (b[:,2] << 2)
    return "".join(str(int(v)) for v in vals)

def show(vec, n=24, label=""):
    s = "".join("1" if x else "0" for x in vec[:n])
    print(f"{label}{s} …")

def debug_l1c_pilot(w: int, p: int, *, expected_octal_prefix: str | None = None, opt: L1cOptions | None = None):
    """
    Print a step-by-step view and a few octal variants to pinpoint mismatches.
    """
    if opt is None:
        opt = L1cOptions()

    print("=== Parameters ===")
    print(opt)
    print(f"w={w}, p={p}")
    print("Building sequences…")

    k = opt.k
    resid01 = residue_indicator(k, map_zero_to_one=opt.map_zero_to_one)
    show(resid01, 24, "resid01[0:24]      = ")

    resid01_shift = np.roll(resid01, (w % k) if opt.shift_direction=="right" else -(w % k))
    show(resid01_shift, 24, "resid01_shift    = ")

    if opt.xor:
        weil01 = (resid01 ^ resid01_shift).astype(np.uint8)
        print("combination        : XOR")
    else:
        weil01 = (1 - (resid01 ^ resid01_shift)).astype(np.uint8)
        print("combination        : XNOR")

    show(weil01, 24, "weil01[0:24]      = ")

    start = (p - opt.p_base) % k
    win = np.concatenate((weil01[start:start+24],))  # peek window
    show(win, 24, f"window@start={start} = ")

    out01 = build_l1c_pilot(w, p, opt)
    show(out01, 48, "out01[0:48]       = ")

    oct_std   = bits01_to_octal(out01, msb_first=True,  reverse_bits_in_group=False)
    oct_rev   = bits01_to_octal(out01, msb_first=True,  reverse_bits_in_group=True)
    oct_lsb   = bits01_to_octal(out01, msb_first=False, reverse_bits_in_group=False)
    oct_lsb_r = bits01_to_octal(out01, msb_first=False, reverse_bits_in_group=True)

    print("\nOctal (first 32 digits, different packing conventions):")
    print("MSB-first, group normal : ", oct_std[:32])
    print("MSB-first, group reversed: ", oct_rev[:32])
    print("LSB-first, group normal : ", oct_lsb[:32])
    print("LSB-first, group reversed: ", oct_lsb_r[:32])

    if expected_octal_prefix:
        print("\nComparing to expected prefix:")
        candidates = {
            "MSB-first, group normal": oct_std,
            "MSB-first, group reversed": oct_rev,
            "LSB-first, group normal": oct_lsb,
            "LSB-first, group reversed": oct_lsb_r,
        }
        for name, octs in candidates.items():
            match_len = 0
            for a, b in zip(octs, expected_octal_prefix):
                if a == b:
                    match_len += 1
                else:
                    break
            print(f"{name:27s} match {match_len} octal digits")


In [25]:
PRN = 1
w_pilot = l1c_table.at[PRN, "w_pilot"]
p_pilot = l1c_table.at[PRN, "p_pilot"]
print (f"W = {w_pilot}, P={p_pilot}")
opt = L1cOptions(
    k=10223, N0=10230,
    map_zero_to_one=False,   # Legendre indicator: 0 stays 0
    shift_direction="right", # np.roll(+w)
    xor=True,                # XOR for Weil
    p_base=1,                # p is 1-based per ICD tables
    invert=False,            # no global inversion
    octal_msb_first=True,
    reverse_bits_in_group=False
)

# If you know the first N octal digits from the table for this PRN, put them here:
expected = l1c_table.at[PRN, "Initial_pilot_code"]  # e.g., "1732050..."  (leave None if you don't have it handy)

debug_l1c_pilot(w_pilot, p_pilot, expected_octal_prefix=expected, opt=opt)

W = 5111, P=412
=== Parameters ===
L1cOptions(k=10223, N0=10230, map_zero_to_one=False, shift_direction='right', xor=True, p_base=1, invert=False, octal_msb_first=True, reverse_bits_in_group=False)
w=5111, p=412
Building sequences…
resid01[0:24]      = 011110111100101010100101 …
resid01_shift    = 110110000011110100100100 …
combination        : XOR
weil01[0:24]      = 101000111111011110000001 …
window@start=411 = 101000000110101100001100 …
out01[0:48]       = 101000000110101100001100000001000011110101100011 …

Octal (first 32 digits, different packing conventions):
MSB-first, group normal :  50065414010365433711121652067557
MSB-first, group reversed:  50035141040635166744424352037557
LSB-first, group normal :  50035141040635166744424352037557
LSB-first, group reversed:  50065414010365433711121652067557

Comparing to expected prefix:
MSB-first, group normal     match 0 octal digits
MSB-first, group reversed   match 0 octal digits
LSB-first, group normal     match 0 octal digits
LSB-firs

In [26]:
def jacobi(n: int, k: int) -> int:
    assert k > 0 and (k % 2 == 1)
    n = n % k
    t = 1
    while n != 0:
        while n % 2 == 0:
            n //= 2
            r = k % 8
            if r == 3 or r == 5:
                t = -t
        n, k = k, n
        if (n % 4 == 3) and (k % 4 == 3):
            t = -t
        n = n % k
    return t if k == 1 else 0

def l1c_pilot_from_symbols(w: int, p: int, *, k: int = 10223, N0: int = 10230,
                           p_base: int = 1, shift: str = "right",
                           alphabet: str = "01") -> np.ndarray:
    """
    L1C pilot via Weil code constructed from Jacobi/Legendre symbols L(n) in {+1,0,-1}.
    w: Weil index (integer)
    p: insertion point (1-based by default)
    k: prime modulus (10223)
    N0: output chips (10230)
    shift: 'right' means use L(n+w); 'left' uses L(n-w)
    alphabet: '01' or 'pm1'
    """
    # Build L(n) for n=0..k-1 with L(0)=0
    idx = np.arange(k, dtype=np.int64)
    vjac = np.vectorize(lambda n: jacobi(int(n), k), otypes=[int])
    L = vjac(idx).astype(np.int8)  # values in {-1,0,+1}; L[0]==0

    # Shifted L
    w_mod = int(w) % k
    Ls = np.roll(L, w_mod if shift == "right" else -w_mod)

    # Weil bits: w(n) = (1 - L(n)*L(n+w)) / 2 in {0,1}
    prod = (L.astype(np.int16) * Ls.astype(np.int16))  # {-1,0,+1}
    weil01 = ((1 - prod) // 2).astype(np.uint8)        # map: +1→0, 0→1, -1→1

    # Extract N0 chips starting at p (1-based default), with wrap
    start = (int(p) - p_base) % k
    if N0 <= k - start:
        out01 = weil01[start:start + N0].copy()
    else:
        tail = k - start
        out01 = np.concatenate((weil01[start:], weil01[:N0 - tail]))

    if alphabet == "01":
        return out01
    elif alphabet == "pm1":
        return (out01 * 2 - 1).astype(np.int8)
    else:
        raise ValueError("alphabet must be '01' or 'pm1'")


In [27]:
chips01 = l1c_pilot_from_symbols(w_pilot, p_pilot, k=10223, N0=10230, p_base=1, shift="right", alphabet="01")

In [28]:
print(chips01[:32])

[1 0 1 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 0]


In [29]:
def bits01_to_octal(bits01: np.ndarray) -> str:
    n = (bits01.size // 3) * 3
    b = bits01[:n].reshape(-1, 3)
    vals = (b[:,0] << 2) | (b[:,1] << 1) | b[:,2]
    return "".join(str(int(v)) for v in vals)

print(bits01_to_octal(chips01)[:32])


50065414010365433711121652067557
